## Run Twitter API to collect tweets

In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import spacy
nlp = spacy.load('en')
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
import nltk


Using TensorFlow backend.


## Loading data from txt file generated through twitter API

### This is a smaller amount of practice data to perfect my code. 

```It also took forever to get the tweets to load without errors```

In [2]:
tweets = pd.read_csv('/Users/macbook/GA-DSI/projects/projects-capstone/captured-tweets.txt', sep='\n', header = None, error_bad_lines=False)

Skipping line 439: expected 1 fields, saw 2
Skipping line 632: expected 1 fields, saw 2
Skipping line 1045: expected 1 fields, saw 2
Skipping line 2276: expected 1 fields, saw 2
Skipping line 2788: expected 1 fields, saw 2
Skipping line 2943: expected 1 fields, saw 2
Skipping line 3259: expected 1 fields, saw 2
Skipping line 3285: expected 1 fields, saw 2
Skipping line 3376: expected 1 fields, saw 2
Skipping line 3405: expected 1 fields, saw 2
Skipping line 3427: expected 1 fields, saw 2
Skipping line 3751: expected 1 fields, saw 2



In [3]:
tweets.head(10)

,0
0,RT @HENDRIXNASH: Premiered by @NoiseyMusic. - ...
1,RT @GamingAnarchist: https://t.co/ByrvubKPH4
2,This is literally the same discredited Jack Th...
3,http…
4,When/if Presidential election coverage returns...
5,Trump supporters really need to grasp that the...
6,RT @krglick: Congrats to my good friend @plwil...
7,RT @tagaq: @julytalk our sister debilitated wa...
8,RT @billboard: Watch Miranda Lambert rock out ...
9,RT @GamingAnarchist: https://t.co/ByrvubKPH4


## Cleaning data and preparing for NLP
Each area specified as subject will be switched out for specific subject to be run

In [4]:
# removing punctuation
import string

def normalize_string(s):
    for p in string.punctuation:
        s = s.replace(p, ' ')
    s = s.lower().strip()
    return s

In [5]:
# Changing column name from 0 to original tweets
tweets['original_tweet'] = tweets[0]
del tweets[0]

In [6]:
tweets['original_tweet'] = tweets['original_tweet'].apply(normalize_string)

In [7]:
tweets.head()

,original_tweet
0,rt hendrixnash premiered by noiseymusic ...
1,rt gaminganarchist https t co byrvubkph4
2,this is literally the same discredited jack th...
3,http…
4,when if presidential election coverage returns...


In [8]:
# create column for show subject using branded hashtags
# Create column for updates
def find_bronson(x):
    res = re.search('(trump)', x)
    if res:
        return res.group()
    else:
        return pd.np.nan

In [9]:
tweets['bronson'] = tweets['original_tweet'].apply(find_bronson)

In [10]:
tweets['bronson'].value_counts

<bound method Series.value_counts of 0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
5       trump
6         NaN
7         NaN
8         NaN
9         NaN
10        NaN
11        NaN
12        NaN
13        NaN
14        NaN
15        NaN
16        NaN
17        NaN
18        NaN
19        NaN
20        NaN
21      trump
22        NaN
23      trump
24      trump
25        NaN
26        NaN
27        NaN
28      trump
29        NaN
        ...  
3595      NaN
3596      NaN
3597      NaN
3598      NaN
3599      NaN
3600      NaN
3601      NaN
3602      NaN
3603      NaN
3604      NaN
3605    trump
3606      NaN
3607    trump
3608    trump
3609      NaN
3610      NaN
3611      NaN
3612      NaN
3613    trump
3614      NaN
3615    trump
3616      NaN
3617      NaN
3618      NaN
3619      NaN
3620      NaN
3621      NaN
3622      NaN
3623      NaN
3624      NaN
Name: bronson, dtype: object>

In [11]:
tweets.head(10)

,original_tweet,bronson
0,rt hendrixnash premiered by noiseymusic ...,NaN
1,rt gaminganarchist https t co byrvubkph4,NaN
2,this is literally the same discredited jack th...,NaN
3,http…,NaN
4,when if presidential election coverage returns...,NaN
5,trump supporters really need to grasp that the...,trump
6,rt krglick congrats to my good friend plwil...,NaN
7,rt tagaq julytalk our sister debilitated wa...,NaN
8,rt billboard watch miranda lambert rock out ...,NaN
9,rt gaminganarchist https t co byrvubkph4,NaN


## Following codes don't completely work yet

In [12]:
# Df with only tweets containing subject
df_bronson = tweets[tweets['bronson'] == 'trump']
df_bronson

,original_tweet,bronson
5,trump supporters really need to grasp that the...,trump
21,rt iamrashidajones trump supporters really n...,trump
23,rt iamrashidajones trump supporters really n...,trump
24,when trumppence16 becomes potus2016 hopefull...,trump
28,rt iamrashidajones trump supporters really n...,trump
31,rt iamrashidajones trump supporters really n...,trump
35,rt iamrashidajones trump supporters really n...,trump
39,rt mradamscott impossible to overstate how i...,trump
41,rt mradamscott impossible to overstate how i...,trump
44,rt mradamscott impossible to overstate how i...,trump


In [13]:
df_bronson.head()

,original_tweet,bronson
5,trump supporters really need to grasp that the...,trump
21,rt iamrashidajones trump supporters really n...,trump
23,rt iamrashidajones trump supporters really n...,trump
24,when trumppence16 becomes potus2016 hopefull...,trump
28,rt iamrashidajones trump supporters really n...,trump


In [14]:

df_bronson['original_tweet'] = df_bronson.loc[:,('original_tweet')].map(lambda x: x.decode('unicode_escape').encode('ascii', 'ignore').strip())

/Users/macbook/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [15]:
df_bronson_o = df_bronson[['original_tweet']]

In [20]:
df_bronson_sp = " ".join(df_bronson_o.as_matrix().flatten())

In [ ]:
df_bronson.to_csv('capstone_test1.csv')

In [ ]:
def count_entity_sentiment(nlp, texts):
    '''Compute the net document sentiment for each entity in the texts.'''
    entity_sentiments = collections.Counter(float)
    for doc in nlp.pipe(texts, batch_size=1000, n_threads=4):
        for ent in doc.ents:
            entity_sentiments[ent.text] += doc.sentiment
    return entity_sentiments

In [21]:
import spacy                         # See "Installing spaCy"
nlp = spacy.load('en')               # You are here.
doc = nlp(df_brondon_sp)         


NameError: name 'df_brondon_sp' is not defined

In [ ]:
def read_file(file_name):
    with open(read_file, 'r') as file:
        return file.read().decode('utf-8')

In [ ]:
def read_file(file_name):
    with open(read_file, 'r') as file:
        return file.read()

In [ ]:
import spacy                         # See "Installing spaCy"
nlp = spacy.load('en')     
text = read_file('/Users/macbook/GA-DSI/projects/projects-capstone/captured-tweets.txt')
processed_text = nlp(text)

In [ ]:
# tokenized  and stemmed subject
df_bronson['tokenized_bronson'] = df_bronson.apply(lambda row: nltk.word_tokenize(row['original_tweet']), axis=1)
a = df_bronson.loc[:,'tokenized_bronson']
lancaster = nltk.LancasterStemmer()
stems = [lancaster.stem(i) for i in a]
print stems

In [ ]:
# Import the stop word list
from nltk.corpus import stopwords 
stop = stopwords.words('english')

df_bronson['tokenized_bronson'] = df_bronson['tokenized_bronson'].apply(lambda x: [item for item in x if item not in stop])


stop = stopwords.words('english')

### Attempting LDA. Possibly Abandoning

In [ ]:
from gensim import corpora, models

dictionary = corpora.Dictionary(tweets)


### Just got tensorflow fully up and running Thursday night. Practicing models via Keras.

In [ ]:
tweets.head()

In [ ]:
# Keras Model for guidance and reference
# for a single-input model with 2 classes (binary):

model = Sequential()
model.add(Dense(1, input_dim=784, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# generate dummy data
import numpy as np
data = np.random.random((1000, 784))
labels = np.random.randint(2, size=(1000, 1))

# train the model, iterating on the data in batches
# of 32 samples
model.fit(data, labels, nb_epoch=10, batch_size=32)

## Once data is finished, I am planning on using Naive Bayse

### Below this point are code snippets that I am not using yet but are possible

In [ ]:


tweets_data = []
tweets_file = tweets
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [ ]:
def get_words_in_tweets(df):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
        return all_words


In [ ]:
get_words_in_tweets(tweets)

In [ ]:
from textblob import TextBlob
sentiment = [i for i in tweets]
sentiment

In [ ]:
# Spacy deep learning code. Not mine. Here for guidance

def count_entity_sentiment(nlp, texts):
    '''Compute the net document sentiment for each entity in the texts.'''
    entity_sentiments = collections.Counter(float)
    for doc in nlp.pipe(texts, batch_size=1000, n_threads=4):
        for ent in doc.ents:
            entity_sentiments[ent.text] += doc.sentiment
    return entity_sentiments

def load_nlp(lstm_path, lang_id='en'):
    def create_pipeline(nlp):
        return [nlp.tagger, nlp.entity, SentimentAnalyser.load(lstm_path, nlp)]
    return spacy.load(lang_id, create_pipeline=create_pipeline)